In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

# Indep variable cleaning

In [2]:
multifamily = {'R-M', 'R-M(PD)', 'R-M(CL)', 'R-2', 'R-2(PD)', 'UV', 'CP', 
               'CP(PD)', 'MUC', 'MUN', 'DC', 'DC-NT1', 'DC(PD)'}
#confirm what R-M(CL) means
#make sure we don't need to include others

In [17]:
sj_zips = gpd.read_file('Zip_Code_Boundary.geojson')
sj_zips = sj_zips.to_crs(epsg = '3857') #converting crs to planar


In [16]:
sj_zones = gpd.read_file('zoning_data/Zoning_2022.geojson')
sj_zones = sj_zones.to_crs(epsg = '3857') #converting crs to planar


In [18]:
sj_zones['area'] = sj_zones.area #creating area column

In [12]:
#check number of parcels between years
# calculate size from geometry
# check change in other zoning proportions
# urban institute yonah freemark

# capture pressure on housing
# -> 
# base rent and total housing units in log
# include base proportion of multifamily

In [19]:
#only one row where the zoningabbrev != zoning
sj_zones[sj_zones['ZONING'] != sj_zones['ZONINGABBREV']]

,OBJECTID,FACILITYID,ZONING,ZONINGABBREV,REZONINGFILE,PDUSE,PDDENSITY,DEVELOPEDASPD,APPROVALDATE,COLORCODE,LASTUPDATE,NOTES,SHAPE_Length,SHAPE_Area,INTID,geometry,area
12484,27387,13466,R-M(CL),R-M,NaN,NaN,0,NaN,NaN,3,2022-10-11 21:49:45+00:00,NaN,1021.196388,41163.034866,13466,"POLYGON ((-13566174.363 4484780.976, -13566204...",6059.180697


In [15]:
sj_zones['ZONING'].unique()

array(['A', 'R-1-5', 'IP', 'R-2(PD)', 'R-1-8', 'PQP', 'OS', 'R-M',
       'A(PD)', 'CN', 'R-1-1', 'LI(PD)', 'CP', 'TR', 'R-M(PD)', 'MUC',
       'CN(PD)', 'TEC', 'CIC', 'UR', 'UV', 'CG', 'UVC', 'DC', 'R-2',
       'CP(PD)', 'CG(PD)', 'MUN', 'CIC(PD)', 'CO', 'CO(PD)', 'LI',
       'DC-NT1', 'HI', 'HI(PD)', 'IP(PD)', 'OS(PD)', 'R-1-1(PD)', 'R-1-2',
       'R-1-2(PD)', 'R-1-5(CL)', 'R-1-5(PD)', 'R-1-8(CL)', 'R-1-8(PD)',
       'R-1-RR', 'R-M(CL)', 'R-MH', 'WATER', 'PQP(PD)', 'DC(PD)',
       'TEC(PD)', 'MS-C', 'MS-G'], dtype=object)

In [20]:
#should this be an inner join or left/right
joined_data = gpd.sjoin(sj_zips, sj_zones, how='inner', predicate='intersects')

In [8]:
#joined_data[['ZIPCODE', 'FACILITYID', 'ZONING', 'ZONINGABBREV']]

In [21]:
def calculate_percentage(group):
    total_area = group['area'].sum()
    rm_area = group[group['ZONINGABBREV'].isin(multifamily)]['area'].sum()
    percentage = rm_area / total_area
    return percentage

In [22]:
percentages = (
    joined_data.groupby('ZIPCODE').apply(calculate_percentage)
    .reset_index(name='multifam_percentage')
)

In [24]:
joined_data = joined_data.merge(percentages, on = 'ZIPCODE', how = 'left')

In [25]:
#facilityID/objectID might be the unique ID
joined_data.head()

,OBJECTID_left,ZIPCODE,LASTUPDATE_left,SHAPE_Length_left,SHAPE_Area_left,SANJOSELIMITS,geometry,index_right,OBJECTID_right,FACILITYID,...,DEVELOPEDASPD,APPROVALDATE,COLORCODE,LASTUPDATE_right,NOTES,SHAPE_Length_right,SHAPE_Area_right,INTID,area,multifam_percentage
0,2448,95113,2021-08-05 16:20:58+00:00,44802.535193,1.214197e+07,Yes,"MULTIPOLYGON (((-13567772.444 4486736.782, -13...",11662,11843,11843,...,NaN,NaN,3,2022-04-22 21:50:37+00:00,NaN,1416.945085,108790.575196,11843,16012.548031,0.655577
1,5267,95110,2021-08-05 16:25:43+00:00,125080.893238,1.272895e+08,Yes,"MULTIPOLYGON (((-13568165.458 4485007.913, -13...",11662,11843,11843,...,NaN,NaN,3,2022-04-22 21:50:37+00:00,NaN,1416.945085,108790.575196,11843,16012.548031,0.165249
2,2448,95113,2021-08-05 16:20:58+00:00,44802.535193,1.214197e+07,Yes,"MULTIPOLYGON (((-13567772.444 4486736.782, -13...",11673,11854,11854,...,NaN,NaN,3,2022-04-22 21:50:43+00:00,NaN,1367.437248,76529.248044,11854,11264.828469,0.655577
3,5267,95110,2021-08-05 16:25:43+00:00,125080.893238,1.272895e+08,Yes,"MULTIPOLYGON (((-13568165.458 4485007.913, -13...",11673,11854,11854,...,NaN,NaN,3,2022-04-22 21:50:43+00:00,NaN,1367.437248,76529.248044,11854,11264.828469,0.165249
4,2448,95113,2021-08-05 16:20:58+00:00,44802.535193,1.214197e+07,Yes,"MULTIPOLYGON (((-13567772.444 4486736.782, -13...",7333,7463,7463,...,NaN,NaN,8,2023-03-13 23:58:45+00:00,NaN,787.447414,32513.751320,7463,4785.660465,0.655577


In [26]:
#keep all rows from the left and duplicate them if necessary to represent multiple hits between the two dataframes.
#print(sj_zones.shape[0])
#joined_data.shape[0]

In [27]:
#i think facilityID/objectID is the zoning district, confirm this
abbrev_data = joined_data[['ZIPCODE', 'FACILITYID', 'ZONINGABBREV', 'multifam_percentage']]

In [28]:
abbrev_data['ZIPCODE'] = abbrev_data['ZIPCODE'].astype('int64')

/var/folders/dl/8n__h2151hd3n3gtfckwb75w0000gn/T/ipykernel_5565/1191412562.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abbrev_data['ZIPCODE'] = abbrev_data['ZIPCODE'].astype('int64')


In [29]:
base_zones = gpd.read_file('zoning_data/Zoning_2012.geojson')

In [30]:
#is this the right crs?
base_zones['geometry'] = base_zones['geometry'].to_crs(crs = 4326)

In [31]:
base_joined = gpd.sjoin(sj_zips, base_zones, how='inner', predicate='intersects')

base_percentages = (
    base_joined.groupby('ZIPCODE').apply(calculate_percentage)
    .reset_index(name='multifam_percentage_2012')
)

base_joined = base_joined.merge(base_percentages, on = 'ZIPCODE', how = 'left')

abbrev_base = base_joined[['ZIPCODE', 'FACILITYID', 'ZONINGABBREV', 'multifam_percentage_2012']]
abbrev_base['ZIPCODE'] = abbrev_base['ZIPCODE'].astype('int64')

/var/folders/dl/8n__h2151hd3n3gtfckwb75w0000gn/T/ipykernel_5565/2301465851.py:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:3857
Right CRS: EPSG:4326

  base_joined = gpd.sjoin(sj_zips, base_zones, how='inner', predicate='intersects')
/var/folders/dl/8n__h2151hd3n3gtfckwb75w0000gn/T/ipykernel_5565/2301465851.py:4: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  base_joined.groupby('ZIPCODE').apply(calculate_percentage)


TypeError: DataFrame.reset_index() got an unexpected keyword argument 'name'

In [126]:
zones_15 = gpd.read_file('zoning_data/Zoning_2015.geojson')

In [127]:
#is this the right crs?
zones_15 = zones_15.to_crs(sj_zones.crs)

joined_15 = gpd.sjoin(sj_zips, zones_15, how='inner', predicate='intersects')

percentages_2015 = (
    joined_15.groupby('ZIPCODE').apply(calculate_percentage)
    .reset_index(name='multifam_percentage_2015')
)

joined_15 = joined_15.merge(percentages_2015, on = 'ZIPCODE', how = 'left')

abbrev_15 = joined_15[['ZIPCODE', 'FACILITYID', 'ZONINGABBREV', 'multifam_percentage_2015']] 
abbrev_15['ZIPCODE'] = abbrev_15['ZIPCODE'].astype('int64')
#i think it should be objectid

/var/folders/dl/8n__h2151hd3n3gtfckwb75w0000gn/T/ipykernel_93981/909249531.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  abbrev_15['ZIPCODE'] = abbrev_15['ZIPCODE'].astype('int64')


In [128]:
to_join_base = abbrev_base[['ZIPCODE', 'multifam_percentage_2012']].groupby('ZIPCODE').max()
to_join_15 = abbrev_15[['ZIPCODE', 'multifam_percentage_2015']].groupby('ZIPCODE').max()
to_join_final = abbrev_data[['ZIPCODE', 'multifam_percentage']].groupby('ZIPCODE').max()

In [129]:
temp = to_join_base.merge(to_join_15, how = 'inner', left_index = True, right_index = True)

In [130]:
zip_panel = temp.merge(to_join_final, how = 'inner', left_index = True, right_index = True)

In [131]:
zip_panel['intial_diff'] = zip_panel['multifam_percentage_2015'] - zip_panel['multifam_percentage_2012']
zip_panel['final_diff'] = zip_panel['multifam_percentage'] - zip_panel['multifam_percentage_2015']
zip_panel

,multifam_percentage_2012,multifam_percentage_2015,multifam_percentage,intial_diff,final_diff
ZIPCODE,,,,,
94089,0.000000,0.000000,0.000000,0.000000,0.000000
95002,9.302326,9.302326,14.285714,0.000000,4.983389
95008,30.666667,32.432432,27.659574,1.765766,-4.772858
95013,0.000000,0.000000,0.000000,0.000000,0.000000
95014,28.571429,30.769231,33.333333,2.197802,2.564103
95032,26.666667,26.666667,6.976744,0.000000,-19.689922
95035,12.000000,12.000000,0.000000,0.000000,-12.000000
95037,2.127660,2.127660,0.000000,0.000000,-2.127660
95050,12.500000,12.500000,19.230769,0.000000,6.730769


In [133]:
#abbrev_data['ZONINGABBREV'].value_counts()

# Rent data

In [180]:
rents = pd.read_excel('fy2023_safmrs_revised.xlsx')

In [181]:
rents.rename({
    'ZIP\nCode': 'ZIP Code', 
    'SAFMR\n0BR': 'SAFMR 0BR',
    'SAFMR\n0BR - 90%\nPayment\nStandard': 'SAFMR 0BR - 90% Payment Standard',
    'SAFMR\n0BR - 110%\nPayment\nStandard': 'SAFMR 0BR - 110% Payment Standard',
    'SAFMR\n1BR': 'SAFMR 1BR',
       'SAFMR\n1BR - 90%\nPayment\nStandard': 'SAFMR 1BR - 90% Payment Standard',
       'SAFMR\n1BR - 110%\nPayment\nStandard': 'SAFMR 1BR - 110% Payment Standard', 
    'SAFMR\n2BR': 'SAFMR 2BR',
       'SAFMR\n2BR - 90%\nPayment\nStandard': 'SAFMR 2BR - 90% Payment Standard',
       'SAFMR\n2BR - 110%\nPayment\nStandard': 'SAFMR 2BR - 110% Payment Standard', 
    'SAFMR\n3BR': 'SAFMR 3BR',
       'SAFMR\n3BR - 90%\nPayment\nStandard': 'SAFMR 3BR - 90% Payment Standard',
       'SAFMR\n3BR - 110%\nPayment\nStandard': 'SAFMR 3BR - 110% Payment Standard', 
    'SAFMR\n4BR': 'SAFMR 4BR',
       'SAFMR\n4BR - 90%\nPayment\nStandard': 'SAFMR 4BR - 90% Payment Standard',
       'SAFMR\n4BR - 110%\nPayment\nStandard': 'SAFMR 4BR - 110% Payment Standard'
}, axis = 1, inplace = True)

In [182]:
#rents.head()

In [183]:
abbrev_rents = rents[['ZIP Code', 'SAFMR 0BR', 'SAFMR 1BR',
                     'SAFMR 2BR', 'SAFMR 3BR', 'SAFMR 4BR']]
del rents

In [184]:
abbrev_rents.head()

,ZIP Code,SAFMR 0BR,SAFMR 1BR,SAFMR 2BR,SAFMR 3BR,SAFMR 4BR
0,76437,620,660,860,1160,1440
1,76443,620,660,860,1160,1440
2,76464,620,660,860,1160,1440
3,76469,620,660,860,1160,1440
4,79501,660,700,900,1220,1480


In [202]:
rents_2012 = pd.read_excel('small_area_fmrs_fy2012.xls')

In [206]:
rents_2012 = rents_2012[rents_2012['County Name'] == 'Santa Clara County']

In [207]:
rents_2012 = rents_2012[['ZIP', 'area_rent_br0', 'area_rent_br1', 'area_rent_br2', 'area_rent_br3', 'area_rent_br4']]

In [208]:
rents_2012.rename({
    'area_rent_br0': '0BR_base', 'area_rent_br1': '1BR_base', 
    'area_rent_br2': '2BR_base', 'area_rent_br3': '3BR_base', 
    'area_rent_br4': '4BR_base'
}, axis = 1, inplace = True)

In [209]:
rents_2012['ZIP'] = rents_2012['ZIP'].astype('int64')

In [211]:
merged_rents = rents_2012.merge(abbrev_rents, left_on = 'ZIP', right_on = 'ZIP Code')

In [212]:
merged_rents

,ZIP,0BR_base,1BR_base,2BR_base,3BR_base,4BR_base,ZIP Code,SAFMR 0BR,SAFMR 1BR,SAFMR 2BR,SAFMR 3BR,SAFMR 4BR
0,94020,1080,1470,1630,2310,2860,94020,1980,2380,2910,3710,4040
1,94020,1080,1470,1630,2310,2860,94020,1980,2380,2910,3710,4040
2,94022,1080,1470,1630,2310,2860,94022,3160,3570,4180,5330,5970
3,94024,1520,2060,2290,3240,4020,94024,3110,3510,4110,5240,5870
4,94028,1080,1470,1630,2310,2860,94028,2390,2850,3380,4210,4660
...,...,...,...,...,...,...,...,...,...,...,...,...
71,95138,1480,2000,2230,3160,3910,95138,2530,2860,3350,4270,4790
72,95139,1260,1710,1900,2690,3330,95139,2670,3020,3530,4500,5040
73,95140,1080,1470,1630,2310,2860,95140,2110,2440,2910,3750,4300
74,95141,1080,1470,1630,2310,2860,95141,2220,2510,2940,3750,4200


In [220]:
merged_rents['0BR_change'] = merged_rents['SAFMR 0BR'] - merged_rents['0BR_base']
merged_rents['1BR_change'] = merged_rents['SAFMR 1BR'] - merged_rents['1BR_base']
merged_rents['2BR_change'] = merged_rents['SAFMR 2BR'] - merged_rents['2BR_base']
merged_rents['3BR_change'] = merged_rents['SAFMR 3BR'] - merged_rents['4BR_base']
merged_rents['4BR_change'] = merged_rents['SAFMR 4BR'] - merged_rents['4BR_base']

changes = merged_rents[['ZIP', '0BR_change', '1BR_change', 
                        '2BR_change', '3BR_change', '4BR_change']]

In [224]:
changes['ZIP'].unique()

array([94020, 94022, 94024, 94028, 94040, 94041, 94043, 94085, 94086,
       94087, 94089, 94301, 94303, 94304, 94305, 94306, 94539, 94550,
       95002, 95008, 95013, 95014, 95020, 95023, 95030, 95032, 95033,
       95035, 95037, 95046, 95050, 95051, 95053, 95054, 95070, 95076,
       95110, 95111, 95112, 95113, 95116, 95117, 95118, 95119, 95120,
       95121, 95122, 95123, 95124, 95125, 95126, 95127, 95128, 95129,
       95130, 95131, 95132, 95133, 95134, 95135, 95136, 95138, 95139,
       95140, 95141, 95148])

In [230]:
len(abbrev_data['ZIPCODE'].unique())

42

In [232]:
full = abbrev_data.merge(changes, left_on = 'ZIPCODE', right_on = 'ZIP', how = 'inner')
full.drop('ZIP', axis=1, inplace=True)

In [233]:
full

,ZIPCODE,FACILITYID,ZONINGABBREV,multifam_percentage,0BR_change,1BR_change,2BR_change,3BR_change,4BR_change
0,95113,11843,R-M,4.098361,1950,2050,2450,2580,3130
1,95113,11854,R-M,4.098361,1950,2050,2450,2580,3130
2,95113,7463,MUC,4.098361,1950,2050,2450,2580,3130
3,95113,7473,CP,4.098361,1950,2050,2450,2580,3130
4,95113,7476,CP,4.098361,1950,2050,2450,2580,3130
...,...,...,...,...,...,...,...,...,...
13976,95119,7769,IP,1.652893,1050,890,1150,510,990
13977,95119,7771,IP,1.652893,1050,890,1150,510,990
13978,95119,12099,A(PD),1.652893,1050,890,1150,510,990
13979,95119,6464,A(PD),1.652893,1050,890,1150,510,990


In [46]:
full['multifam_percentage']

0        4.098361
1        4.098361
2        4.098361
3        4.098361
4        4.098361
           ...   
13976    1.652893
13977    1.652893
13978    1.652893
13979    1.652893
13980    1.652893
Name: multifam_percentage, Length: 13981, dtype: float64

In [48]:
sorted(full['ZIPCODE'].unique())

[94089,
 95002,
 95008,
 95013,
 95014,
 95032,
 95035,
 95037,
 95050,
 95054,
 95070,
 95110,
 95111,
 95112,
 95113,
 95116,
 95117,
 95118,
 95119,
 95120,
 95121,
 95122,
 95123,
 95124,
 95125,
 95126,
 95127,
 95128,
 95129,
 95130,
 95131,
 95132,
 95133,
 95134,
 95135,
 95136,
 95138,
 95139,
 95140,
 95141,
 95148]

In [12]:
#2011 rents

In [29]:
#issue!
print('Problem: zip code rent data before 2018 were estimates')
print(len(rents_2011[rents_2011['County_Name'] == 'Santa Clara County']['area_rent_br1'].unique()))

(len(rents_2011[rents_2011['County_Name'] == 'Santa Clara County']['zip'].unique()))

Problem: zip code rent data before 2018 were estimates
29


63

# Demographic info (controls)

In [104]:
test_demo = pd.read_csv('controls_data/full_2012_demographics.csv')

In [105]:
test_demo['Label (Grouping)'] = test_demo['Label (Grouping)'].apply(lambda x: str(x).replace(u'\xa0', u''))
test_demo.columns = test_demo.columns.str.replace('!!', ' ')


In [106]:
test_demo['Label (Grouping)'] = test_demo['Label (Grouping)'].apply(lambda x: str(x).replace(u'\xa0', u''))
test_demo.columns = test_demo.columns.str.replace('!!', ' ')


og = test_demo.copy()

#to get the rows outside of "one race" dropdown
rows = [2] + list(range(60, 70))
test_demo = test_demo.iloc[rows]


#note that median age and total housing units are in 'estimate' columns
vals_to_keep = {'Male', 'White', 'Black or African American',
       'American Indian and Alaska Native', 'Asian', 'Native Hawaiian and Other Pacific Islander',
                'Some Other Race', 'Hispanic or Latino (of any race)',
               }

test_demo = test_demo[test_demo['Label (Grouping)'].isin(vals_to_keep)]


test_demo = test_demo.filter(regex='^(?=.*(?i)percent|label)(?!.*(?i)percent margin of error)')

pivoted = test_demo.set_index('Label (Grouping)').T
pivoted.reset_index(inplace = True)
pivoted.rename({'index': 'ZIP'}, axis = 1, inplace = True)
pivoted['ZIP'] = pivoted['ZIP'].str.extract(r'ZCTA5 (\d+) Percent')[0]

In [107]:
og = og.filter(regex='^(?=.*(?i)estimate|label)(?!.*(?i)estimate margin of error)')

og = og.set_index('Label (Grouping)')
og_pivot = og[(og.index == 'Total housing units') | (og.index == 'Median age (years)') | 
              (og.index == 'Total population')].T

og_pivot = og_pivot.T.drop_duplicates(keep = 'first').T

og_pivot.reset_index(inplace = True)
og_pivot.rename({'index': 'ZIP'}, axis = 1, inplace = True)
og_pivot['ZIP'] = og_pivot['ZIP'].str.extract(r'ZCTA5 (\d+) Estimate')[0]

full_controls = pivoted.merge(og_pivot, on = 'ZIP')

In [101]:
def clean_totals(val):
    if isinstance(val, str):
        return float(val.replace(',', ''))
    else:
        return val

In [103]:
#cleaning

full_controls.replace('-', '0', inplace = True)

def clean_pct(column):
    return column.str.rstrip('%').astype(float) / 100


to_clean = ['Male', 'White', 'Black or African American', 'American Indian and Alaska Native',
           'Asian', 'Native Hawaiian and Other Pacific Islander', 'Hispanic or Latino (of any race)']


full_controls[to_clean] = full_controls[to_clean].apply(clean_pct)

full_controls['Total population'] = full_controls['Total population'].apply(clean_totals)

full_controls['Total housing units'] = full_controls['Total housing units'].apply(clean_totals)



full_controls['Density per unit'] = full_controls['Total population'] / full_controls['Total housing units']
full_controls

Label (Grouping),ZIP,Male,White,Black or African American,American Indian and Alaska Native,Asian,Native Hawaiian and Other Pacific Islander,Hispanic or Latino (of any race),Total population,Median age (years),Total housing units,Density per unit
0,94089,0.485,0.477,0.026,0.014,0.441,0.001,0.209,20282.0,36.2,7676.0,2.642262
1,95002,0.491,0.547,0.013,0.010,0.120,0.000,0.709,1906.0,35.6,577.0,3.303293
2,95008,0.493,0.750,0.029,0.015,0.201,0.008,0.164,44909.0,37.8,19203.0,2.338645
3,95013,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0,0.0,NaN
4,95014,0.490,0.349,0.006,0.006,0.643,0.004,0.051,60690.0,39.7,22269.0,2.725313
5,95032,0.480,0.851,0.013,0.016,0.149,0.002,0.066,25039.0,43.3,10433.0,2.399981
6,95035,0.511,0.243,0.036,0.011,0.653,0.012,0.156,67380.0,36.7,20253.0,3.326915
7,95037,0.484,0.739,0.034,0.014,0.127,0.010,0.357,45924.0,37.7,15598.0,2.944224
8,95050,0.507,0.623,0.043,0.013,0.219,0.009,0.325,36587.0,34.0,14487.0,2.525506
9,95054,0.515,0.295,0.056,0.002,0.638,0.005,0.113,23188.0,33.4,8637.0,2.684728


In [58]:
all_zips = set(
[94089,
 95002,
 95008,
 95013,
 95014,
 95032,
 95035,
 95037,
 95050,
 95054,
 95070,
 95110,
 95111,
 95112,
 95113,
 95116,
 95117,
 95118,
 95119,
 95120,
 95121,
 95122,
 95123,
 95124,
 95125,
 95126,
 95127,
 95128,
 95129,
 95130,
 95131,
 95132,
 95133,
 95134,
 95135,
 95136,
 95138,
 95139,
 95140,
 95141,
 95148])

In [59]:
#in 2021 data
all_zips.difference(set(pivoted['ZIP'].astype(int)))

{95141}

In [1]:
#concat median age, sex ratio, total housing units (?)

In [ ]:
#keep median age, all ethnic groups, sex ratio

**Median income**

In [17]:
raw_med_12 = pd.read_csv('controls_data/sj_med_income_2012.csv')

In [18]:
raw_med_12.columns = raw_med_12.columns.str.replace('!!', ' ')

In [53]:
mean_med = raw_med_12[(raw_med_12['Label (Grouping)'] == 'Median income (dollars)') 
           | (raw_med_12['Label (Grouping)'] == 'Mean income (dollars)')]

In [54]:
mean_med = mean_med.filter(regex=r'^(ZCTA5 \d{5} Households Estimate|Label \(Grouping\))$')


In [55]:
mean_med

,Label (Grouping),ZCTA5 94089 Households Estimate,ZCTA5 95002 Households Estimate,ZCTA5 95008 Households Estimate,ZCTA5 95013 Households Estimate,ZCTA5 95014 Households Estimate,ZCTA5 95032 Households Estimate,ZCTA5 95035 Households Estimate,ZCTA5 95037 Households Estimate,ZCTA5 95050 Households Estimate,...,ZCTA5 95131 Households Estimate,ZCTA5 95132 Households Estimate,ZCTA5 95133 Households Estimate,ZCTA5 95134 Households Estimate,ZCTA5 95135 Households Estimate,ZCTA5 95136 Households Estimate,ZCTA5 95138 Households Estimate,ZCTA5 95139 Households Estimate,ZCTA5 95140 Households Estimate,ZCTA5 95148 Households Estimate
11,Median income (dollars),"80,625","83,125","83,306",-,"125,168","122,512","99,527","93,604","65,371",...,"102,954","93,896","67,444","104,814","113,806","87,564","144,912","95,000","162,167","105,577"
12,Mean income (dollars),"96,423","94,243","105,661",-,"147,786","154,043","113,492","122,241","84,184",...,"118,843","105,020","86,580","119,323","134,791","104,888","191,938","113,708","148,348","121,847"


In [76]:
pivoted_income = mean_med.set_index('Label (Grouping)').T


pivoted_income.reset_index(inplace = True)
pivoted_income.rename({'index': 'ZIP'}, axis = 1, inplace = True)
pivoted_income['ZIP'] = pivoted_income['ZIP'].str.extract(r'ZCTA5 (\d+) Households Estimate')[0]
pivoted_income.replace({'-': np.nan}, inplace = True)

pivoted_income['Median income (dollars)'] = (
    pivoted_income['Median income (dollars)'].str.replace(',', '').astype('float64')
)

pivoted_income['Mean income (dollars)'] = (
    pivoted_income['Mean income (dollars)'].str.replace(',', '').astype('float64')
)

In [78]:
pivoted_income

Label (Grouping),ZIP,Median income (dollars),Mean income (dollars)
0,94089,80625.0,96423.0
1,95002,83125.0,94243.0
2,95008,83306.0,105661.0
3,95013,NaN,NaN
4,95014,125168.0,147786.0
5,95032,122512.0,154043.0
6,95035,99527.0,113492.0
7,95037,93604.0,122241.0
8,95050,65371.0,84184.0
9,95054,115068.0,127989.0
